# **NAME: PARTHA SAKHA PAUL**

# **ROLL: MA23M016**

    CS6910_assignment 2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import numpy as np

# data_dir = '/content/drive/My Drive/nature_12K'
data_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# data_dir = '/content/drive/My Drive/nature_12K/inaturalist_12K'


In [2]:
# Defining transforms for the training, validation, and testing sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(data_dir, transform=transform)
print(len(train_data))
# print(train_data[9998])
# Creating data indices for training and validation splits:
dataset_size = len(train_data)  #9999
indices = list(range(dataset_size)) #[0,1,...,9998]
split = int(np.floor(0.2 * dataset_size))  #1999
print(split)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
# print((val_indices))

# Creating data samplers and loaders
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
# print(type(train_sampler))

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=train_sampler)
# print(len(train_loader))
validation_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=valid_sampler)
# print(*validation_loader)

9999
1999


In [ ]:
activation_functions = {
            'relu': F.relu,
            'gelu': F.gelu,
            'silu': F.silu,
            'mish': F.mish
        }

In [3]:
# defining a class CNN that inherits from nn.Module which is PyTorch's base class for all neural network
class my_CNN(nn.Module):
    def __init__(self, num_classes=10, num_filters=[32, 64, 128, 256, 512], filter_sizes=[3, 3, 3, 3, 3], activation_fn='relu', num_neurons_dense=512):
        super(my_CNN, self).__init__()
        # Initializing class variables
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.activation_fn = activation_functions[activation_fn]
        self.num_neurons_dense = num_neurons_dense

        # Creating convolution layers using ModuleList
        self.conv_layers = nn.ModuleList()
        in_channels = 3  # since, input images are RGB
        for out_channels, kernel_size in zip(num_filters, filter_sizes):
            # a convolution layer with in-out channels and kernel size
            conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
            self.conv_layers.append(conv_layer)
            # Update in_channels for the next layer
            in_channels = out_channels

            
        # find the size needed to flatten the conv layer outputs, initializing the dense layer accordingly
        self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
        # initializing the dense layer from flatten size
        self.dense = nn.Linear(self.flatten_size, num_neurons_dense)
        # output layer: maps from the dense layer to the number of classes.
        self.out = nn.Linear(num_neurons_dense, num_classes)


    def init_flatten_size(self, input_shape):
        # to disable gradient calculations for speed up this computation
        with torch.no_grad():
            # a dummy input tensor of the correct shape
            input_tensor = torch.zeros(1, *input_shape)
            # Forward prop through the conv layers to find output size
            output = self.forward_conv_layers(input_tensor)
            # total number of output features is the flat size needed for the dense layer input
            self.flatten_size = output.numel()

    # a function for sequentially passing input through all convolutional layers and applying activation and pooling
    def forward_conv_layers(self, x):
        # Convolution layer
        for conv in self.conv_layers:
            x = conv(x)  # Convolution operation
            x = self.activation_fn(x)  # Apply the specified activation function
            x = F.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2
        return x

    # forward method defines how the input tensor is transformed through the model
    def forward(self, x):
        # Passing input through the convolution blocks
        x = self.forward_conv_layers(x)
        # Flattening the output for the dense layer
        x = torch.flatten(x, 1)   # Flatten the output for the dense layer
        # Dense layer
        x = self.dense(x)
        x = self.activation_fn(x)
        # Output layer
        x = self.out(x)
        return x

    # Method for training and evaluating the model
    def train_and_evaluate(self, train_loader, validation_loader, n_epochs=10, lr=0.001, device=None):
        if device is None:
            # setting up CUDA if available, otherwise, using CPU
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)  # transferring the model to the selected device (GPU or CPU)

        # Initializing the loss function and optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)  # self.parameters() is a built-in method provided by PyTorch's nn.Module that collects all the learnable parameters of our model

        # Training loop
        for epoch in range(n_epochs):
            self.train()  # Setting model to training mode
            training_loss = 0.0

            # Iterate over the training data
            for images, labels in train_loader:
                # transferring images and labels to the current device (GPU or CPU)
                images, labels = images.to(device), labels.to(device)
                # clearing the gradients of all optimized variables
                optimizer.zero_grad()
                # Forward prop: computing predicted outputs by passing inputs to the model
                outputs = self(images)
                # calculating the loss between predicted and true labels
                loss = loss_function(outputs, labels)
                # Backward prop: computing gradient of the loss with respect to parameters
                loss.backward()
                # optimization step (parameter update)
                optimizer.step()
                # updating running training loss
                training_loss += loss.item() * images.size(0)
            
            # average loss over an epoch
            training_loss /= len(train_loader.sampler)

            # Evaluation phase
            self.eval()  # Setting model to evaluation mode
            # initializing the validation loss for the epoch
            validation_loss = 0.0
            # tracking number of correctly predicted instances
            correct = 0
            # tracking total number of instances
            total = 0
            # disable gradient calculation for validation, saving memory and computations
            with torch.no_grad():
                # iterating over the validation data loader
                for images, labels in validation_loader:
                    # transferring images and labels to the current device (GPU or CPU)
                    images, labels = images.to(device), labels.to(device)
                    # Forward prop: computing predicted outputs by passing inputs to the model
                    outputs = self(images)
                    # calculating the loss between predicted and true labels
                    loss = loss_function(outputs, labels)
                    # updating running validation loss
                    validation_loss += loss.item() * images.size(0)
                    # converting output probabilities to predicted class
                    _, predicted = torch.max(outputs.data, 1)
                    # updating total number of instances
                    total += labels.size(0)
                    # updating correctly predicted instances
                    correct += (predicted == labels).sum().item()

            # average validation loss over an epoch
            validation_loss /= len(validation_loader.sampler)
            # validation accuracy
            val_accuracy = correct / total

            # printing training and validation results
            print(f'Epoch {epoch+1}, Training Loss: {training_loss:.4f}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
            wandb.log({"val_accuracy": val_accuracy * 100, "loss": validation_loss})


# model with parameters
model = my_CNN(num_classes=10,
                  num_filters=[32, 64, 128, 256, 512],
                  filter_sizes=[3, 3, 3, 3, 3],
                  activation_fn='relu',
                  num_neurons_dense=512)

print(model)

# model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)

my_CNN(
  (conv_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (dense): Linear(in_features=25088, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
!pip install wandb

In [4]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key)  #25c2257eaf6c22aa056893db14da4ee2bf0a531a

Enter your API: 25c2257eaf6c22aa056893db14da4ee2bf0a531a


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
sweep_config = {
    'method': 'bayes',  # can be grid, random
    'name' : 'cnn kaggle new',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
            'values': [[32, 64, 128, 256, 512], [64, 64, 64, 64, 64], [512, 256, 128, 64, 32]] 
        },
        'filter_sizes':{
            'values': [[3,5,5,7,7], [7,5,3,3,5], [3,5,7,9,5]]
        },
        'activation_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
#         'dropout': {
#             'values': [0.2, 0.3]
#         },
#         'batch_norm':{
#             'values': ['true','false']
#         },
#         'data_augment': {
#             'values': ['true','false']
#         },
        'num_neurons_dense':{
            'values': [64, 128, 256, 512]
        }
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="Deep_learning_A2")

Create sweep with ID: u2oec5rw
Sweep URL: https://wandb.ai/parthasakhapaul/Deep_learning_A2/sweeps/u2oec5rw


In [6]:
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        run_name = "-ac-"+wandb.config.activation_fn+"-filters-"+str(wandb.config.num_filters)+"-filt_size-"+str(wandb.config.filter_sizes)+"-num_neurons_dense-"+str(wandb.config.num_neurons_dense)
        wandb.run.name=run_name
        # Model training
        model = my_CNN(num_classes=10,
                  num_filters=wandb.config.num_filters,
                  filter_sizes=wandb.config.filter_sizes,
                  activation_fn=wandb.config.activation_fn,
                  num_neurons_dense=wandb.config.num_neurons_dense)
        model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)        

wandb.agent(sweep_id, function=main, count=5)  # to run 5 experiments
wandb.finish()

wandb: Agent Starting Run: 58zih8xy with config:
wandb: 	activation_fn: silu
wandb: 	filter_sizes: [3, 5, 5, 7, 7]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 512
wandb: Currently logged in as: parthasakhapaul. Use `wandb login --relogin` to force relogin


Epoch 1, Training Loss: 2.2342, Validation Loss: 2.2017, Validation Accuracy: 0.1846
Epoch 2, Training Loss: 2.0997, Validation Loss: 2.1229, Validation Accuracy: 0.2391
Epoch 3, Training Loss: 2.0202, Validation Loss: 1.9908, Validation Accuracy: 0.2856
Epoch 4, Training Loss: 1.9657, Validation Loss: 2.0447, Validation Accuracy: 0.2886
Epoch 5, Training Loss: 1.9161, Validation Loss: 1.9780, Validation Accuracy: 0.2896
Epoch 6, Training Loss: 1.8290, Validation Loss: 1.9720, Validation Accuracy: 0.2846
Epoch 7, Training Loss: 1.7512, Validation Loss: 1.9641, Validation Accuracy: 0.3117
Epoch 8, Training Loss: 1.6203, Validation Loss: 2.0138, Validation Accuracy: 0.3142
Epoch 9, Training Loss: 1.4941, Validation Loss: 2.1148, Validation Accuracy: 0.2961
Epoch 10, Training Loss: 1.2559, Validation Loss: 2.2173, Validation Accuracy: 0.3207


loss,█▅▂▃▁▁▁▂▅█
val_accuracy,▁▄▆▆▆▆██▇█
loss,2.21734
val_accuracy,32.06603


wandb: Agent Starting Run: o704gdqv with config:
wandb: 	activation_fn: silu
wandb: 	filter_sizes: [7, 5, 3, 3, 5]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 64


Epoch 1, Training Loss: 2.2674, Validation Loss: 2.2142, Validation Accuracy: 0.1671
Epoch 2, Training Loss: 2.1530, Validation Loss: 2.1227, Validation Accuracy: 0.2166
Epoch 3, Training Loss: 2.0611, Validation Loss: 2.0215, Validation Accuracy: 0.2731
Epoch 4, Training Loss: 1.9738, Validation Loss: 2.0093, Validation Accuracy: 0.2831
Epoch 5, Training Loss: 1.9186, Validation Loss: 1.9640, Validation Accuracy: 0.2981
Epoch 6, Training Loss: 1.8437, Validation Loss: 1.9987, Validation Accuracy: 0.2941
Epoch 7, Training Loss: 1.7484, Validation Loss: 1.9380, Validation Accuracy: 0.3162
Epoch 8, Training Loss: 1.6760, Validation Loss: 2.0956, Validation Accuracy: 0.2981
Epoch 9, Training Loss: 1.5545, Validation Loss: 1.9497, Validation Accuracy: 0.3497
Epoch 10, Training Loss: 1.4150, Validation Loss: 2.0706, Validation Accuracy: 0.3177


loss,█▆▃▃▂▃▁▅▁▄
val_accuracy,▁▃▅▅▆▆▇▆█▇
loss,2.0706
val_accuracy,31.76588


wandb: Agent Starting Run: xkxl5qk6 with config:
wandb: 	activation_fn: silu
wandb: 	filter_sizes: [3, 5, 7, 9, 11]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 64


Traceback (most recent call last):
  File "/tmp/ipykernel_34/2820147712.py", line 7, in main
    model = my_CNN(num_classes=10,
  File "/tmp/ipykernel_34/2571271743.py", line 32, in __init__
    self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
  File "/tmp/ipykernel_34/2571271743.py", line 45, in init_flatten_size
    output = self.forward_conv_layers(input_tensor)
  File "/tmp/ipykernel_34/2571271743.py", line 55, in forward_conv_layers
    x = F.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2
  File "/opt/conda/lib/python3.10/site-packages/torch/_jit_internal.py", line 488, in fn
    return if_false(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py", line 791, in _max_pool2d
    return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
RuntimeError: Given input size: (64x1x1). Calculated output size: (64x0x0). Output size is too small


Run xkxl5qk6 errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/2820147712.py", line 7, in main
    model = my_CNN(num_classes=10,
  File "/tmp/ipykernel_34/2571271743.py", line 32, in __init__
    self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
  File "/tmp/ipykernel_34/2571271743.py", line 45, in init_flatten_size
    output = self.forward_conv_layers(input_tensor)
  File "/tmp/ipykernel_34/2571271743.py", line 55, in forward_conv_layers
    x = F.max_pool2d(x, 2)  # Apply max pooling with a kernel size of 2
  File "/opt/conda/lib/python3.10/site-packages/torch/_jit_internal.py", line 488, in fn
    return if_false(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py", line 791, in _max_pool2d
    return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mod

wandb: Ctrl + C detected. Stopping sweep.
